# Question 4: General Theory/Application

_No need to be verbose, it's not fun for anyone_

1. What part of S**L**A**M** did this project deal with? Why? What does the other part deal with and how would it generally work, given that you only have LIDAR scans, RGB video stream, and noisy pose data for a moving robot?


2. Loop closures play an important role in reducing drift, how would you go about detecting these?


3. Explain the structure of your Jacobian. Is the pose-graph fully connected? Why/Why not?


4. With what you know now, how would you describe and differentiate the SLAM frontend and backend? Why do we need to optimise our poses/map in the first place - where does the noise come from/why?

Ans.1.
This project focuses on the Localization component of the SLAM problem, in which the robot must guess/optimise its poses/trajectories by putting more importance on loop pairs that have been warped because of noise in the odometry data collected from the sensors. This is the backend of the SLAM, where we perform non-linear least squares optimization utilising a variety of approaches such as gauss newton, leveberg marquad, and others. The reason for this is that in both questions 1 and 2, we generated noisy data, such as poses, using odometry and loop restrictions, and then got an optimised prediction of the ground truth trajectory, which is nothing more than the robot's Localization concept.
The other section is about Mapping, which entails producing a map of the environment as well as tracking motion along the trajectory. One method could be to register point clouds from a LIDAR sensor, with Landmarks as part of those points, and then optimise the trajectory based on those Landmarks, i.e. ICP SLAM, while simultaneously registering points to create a map of the environment in a global frame. Another option is to use octomap instead of point cloud, which uses less memory than PCD registration.

Ans 2
The challenge of detecting loop closure is comparable to that of visual place recognition. This is part of the SLAM's data association problem frontend. To detect the same area, concepts such as bag of visual words and feature descriptor matching can be used, resulting to loop closure detection. However, for this job, we already had those and had them processed.

Ans3. 
 The shape of jacobian = (420 x 360)
 Because  jacobian is calculated by partially differentiating residual by independent variable present in the residual and concatenating all of the derivative obtained.
 Hence, following the same logic it's dimension = (3*N) x (3*P); where N = number of odometry edges + number of loop pairs + 1 (for anchor edge) = 119 + 20 + 1 = 140 and p = number of poses/vertices in the graph = 120.
 The diagonal elements in part 2's plot of jacobian (both analytcal and jax) relate to odometry data, whereas off-diagonal entries correspond to loop closure restrictions.
 

Ans 4
a) The backend is responsible for receiving the model's interpretation in the form of a pose graph and choosing which trajectory or graph best describes the measurements expressed as graph constraints. This optimization method is derived from an ML formulation and is defined in a least-squares manner. Graph formation, constraint detection, and graph maintenance are all handled by the frontend.

b) The frontend is in charge of collecting data from the robot's different sensors and pre-processing it to obtain things like initial postures, odometry restrictions, loop closures, landmarks, Lidar depth measurement, and so on. This could be fed into the backend, which would then be in charge of creating a map of the environment while also localising the robot by optimising the trajectory based on many restrictions such as loop closure and Landmarks (orientation through ICP). A map can be created using either PCD (registration from Lidar data) or octomap (probabilistic mapping).

c) It's worth noting that the back-end is completely unaffected by the type of sensors used, the electronics, and a variety of other features of graph formation and maintenance. The front-end architecture, on the other hand, is very sensor-dependent, as we need to know what kind of sensory data will be used in order to create ways for converting this relevant data into the graph that will be sent to the back-end. Furthermore, because the back-least-squares end's optimisation is particularly sensitive to outliers, such as erroneous loop closure connections or wrong constraints in general, it's vital to design a consistent network.

Because of the noise in sensor measured data, which is unavoidably present in data from the physical world, we must optimise our poses. This causes a misalignment between the robot's defined trajectory and the actual trajectory it is following, i.e. ground truth. Finally, incorrect estimation of poses as well as a map of the environment, implying that the trajectory may be drifting. As a result, optimising the poses becomes required.
